In [2]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import keras_tuner as kt

from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential

In [63]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [64]:
with tf.device('/device:CPU:0'):

    # Open velocity data set
    ds = xr.open_dataset('/Users/smata/Downloads/processedData/temperatureData_30min.nc')

    # Reformat to Pandas dataframe
    df = ds['t'].to_dataframe().reset_index()

    df = df.pivot(index = 'time', columns = 'height', values = 't')

    df.columns = [f't_{int(height)}m' for height in df.columns]

    df.insert(0, 'L',    ds.L.values)
    df.insert(0, 'TKE',  ds.TKE.values)

    df.insert(0, 'hour_sin', np.sin(2 * np.pi * ds.hr_day.values / 24))
    df.insert(0, 'hour_cos', np.cos(2 * np.pi * ds.hr_day.values / 24))

    df.insert(0, 'day_sin', np.sin(2 * np.pi * ds.day_yr.values / 365))
    df.insert(0, 'day_cos', np.cos(2 * np.pi * ds.day_yr.values / 365))

    # Standardize data
    windCols      = [col for col in df.columns if col.startswith('t_')]
    stabilityCols = ['TKE', 'L']

    # Define and extract input and output columns
    inputs = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'TKE', 'L', 't_10m']

    X = df[inputs].values
    y = df[windCols].values

    # Create input and output arrays
    split_index = int(0.8 * len(X))

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Create TensorFlow datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset  = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    # Ensure all tensors are of the same shape by batching
    train_dataset = train_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset  = test_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

    # Define the model-building function for hyperparameter tuning
    def build_model(hp):
        model = Sequential()

        # Input layer
        model.add(Dense(units = hp.Int('units_input', min_value = 32, max_value = 512, step = 32),
                        activation = 'relu',
                        input_shape = (X_train.shape[1],)))
        
        # Hidden layers
        for i in range(hp.Int('num_layers', 1, 5)):
            model.add(Dense(units = hp.Int(f'units_{i}', min_value = 32, max_value = 512, step = 32),
                            activation='relu'))

        # Output layer
        model.add(Dense(y_train.shape[1], activation = 'linear'))

        # Compile the model
        model.compile(optimizer = hp.Choice('optimizer', ['adam', 'rmsprop']),
                    loss = 'mse',
                    metrics = ['mae'])
        
        return model

    # Set up Keras Tuner for hyperparameter search
    tuner = kt.RandomSearch(
        build_model,
        objective    = 'val_loss',
        max_trials   = 10,
        executions_per_trial = 1,
        directory    = 'hyperparameter_tuning',
        project_name = 'wind_forecast_nn'
    )

    # Search for the best hyperparameters
    tuner.search_space_summary()

    # Search for the best hyperparameters
    tuner.search(train_dataset,
                validation_data = test_dataset,
                epochs = 50,
                callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)])

    # Retrieve the best model
    best_model = tuner.get_best_models(num_models = 1)[0]

    # Summary of the best model
    best_model.summary()

    # Get the optimal hyperparameters
    best_hp = tuner.get_best_hyperparameters()[0]

    # Train the best model with the optimal batch size and number of epochs
    history = best_model.fit(train_dataset,
                            epochs = best_hp.get('epochs', 100),
                            validation_data = test_dataset)

    # Evaluate the model on the test set
    test_loss, test_mae = best_model.evaluate(test_dataset)
    print(f"Test MAE: {test_mae}")

Trial 5 Complete [00h 00m 08s]
val_loss: 0.01439005509018898

Best val_loss So Far: 0.008501367643475533
Total elapsed time: 00h 13m 23s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
64                |480               |units_input
3                 |4                 |num_layers
288               |384               |units_0
adam              |rmsprop           |optimizer
224               |256               |units_1
384               |288               |units_2
416               |32                |units_3

Epoch 1/50
180/180 [==============================] - 0s 2ms/step - loss: 0.0293 - mae: 0.1211 - val_loss: 0.0559 - val_mae: 0.1970
Epoch 2/50
180/180 [==============================] - 0s 1ms/step - loss: 0.0155 - mae: 0.0952 - val_loss: 0.0250 - val_mae: 0.1284
Epoch 3/50
180/180 [==============================] - 0s 2ms/step - loss: 0.0142 - mae: 0.0920 - val_loss: 0.0952 - val_mae: 0.2569
Epoch 4/50
180/180 [==============================] - 0

KeyboardInterrupt: 